# Data Cleanup

## Necessary Steps

1. Understand the data structure: Look through the database, understand the main structure, i.e., what each row represents, and what columns and types of information are available
2. Merge game file to add in week identifier to the new dataset (games.csv is in '1. Additional Data File')
3. Create column categorizations to filter dataset for relevant purposes
4. Break down receiver into its individual row
5. Create playmaker column, and check id uniqueness - check that one ID corresponds to one name
6. Flag non-relevant plays - add a binary column that flags 1 for run, pass, reception, FG/XP, 0 for all others
7. Add any additional stat needed - reception (for plays that fall under 'reception'), target ('reception', 'pass')
8. Add position based off of the highest stat of a player. Position will be refined later with web scraping
9. Ensure that stat are correctly represented for a given position
10. Verify top 50 stats against the reported ones


## Setup Environment

In [ ]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
pd.set_option('display.max_columns', None)

## 1. Upload Data

In [ ]:
'''
Not sure why your folder names were changes. The working directory of this file is where its source code is located
so you can specify directories relatively
'''

In [ ]:
# Store your files in the same folder as the source code, so you don't have to specify the directory
df_file_2019 = "../../1. Raw-Data/data2019.csv"
df_file_game = "../1. Additional-Data/games.csv"

# Use the convention df for dataframes
df = pd.read_csv(df_file_2019)
df_games = pd.read_csv(df_file_game)

## 2. Add in Weekly Identifier

In [ ]:
'''
Headings need to be more descriptive and properly formatted
Removed unused or unecessary code
No need to create new dataframes, just replace the old one
All set up commans to be placed in the setup section
'''

In [ ]:
df = df.merge(df_games[["game_id", "week"]], on = 'game_id')

## 3. Create Categorization Lists

In [ ]:
'''
Automated creation of most of these
A lot of elements need to be re categorized
'''

### 3.1 Main Lists

In [ ]:
# Create a "key" list that contains elements that will always be needed - game id, week, play type
key = []

# There are so many elements here that are missing here
# touchdown, passing toucdown, rushing touchdown, pass attemps, rush attempts, interceptions, fumble lost, 2 point attempts and conversions and so on
# all elements that contribute to points in fantasy should be in the main lists

pass_play = ['pass_length','pass_location','air_yards']
run_play = ['run_location', 'run_gap']
yard_info = ['yrdln','ydstogo','ydsnet','yards_gained', "fumble_recovery_1_yards", "fumble_recovery_2_yards", "return_yards", ""]
receiver_stats = ["receiver_player_id", "receiver_player_name", "lateral_receiver_player_name", "lateral_receiver_player_name", "yards_after_catch"]
# Two point conv should not be in xp
# xp + fg
xp = ['field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result']

### 3.2 Other Lists

In [ ]:
game_info = ['play_id','game_id','home_team','away_team','posteam','posteam_type', 'defteam', 'side_of_field', 'yardline_100','game_date', "year"]
game_time_info = ['quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'time']

# Keep play type in the main lists
gen_play_info = ['drive', 'sp', 'down', 'goal_to_go','desc','play_type','shotgun','no_huddle','qb_dropback','qb_kneel','qb_spike','qb_scramble',]

# automate - see below for best example
timeout_info = ['home_timmouts_remaining','away_timeouts_remaining','timeout','timeout_team']

team_info = ["return_team", 'td_team', 'posteam_time', 'defteam_time', 'total_home_score','total_away_score', 'posteam_score_post','defteam_score_post', 'score_differential', "forced_fumble_player_1_team", "forced_fumble_player_2_team", "solo_tackle_1_team", "solo_tackle_2_team", "assist_tackle_1_team", "assist_tackle_2_team", "assist_tackle_3_team", "assist_tackle_4_team", "fumbled_1_team", "fumbled_2_team", "fumble_recovery_1_team", "fumble_recovery_2_team"]

# automate
probability_info = ['no_score_prob','opp_fg_prob', 'opp_safety_prob', 'opp_td_prob', 'fg_prob', 'safety_prob', 'td_prob', 'extra_point_prob', 'two_point_conversion_prob', 'ep', 'epa', 'total_home_epa','total_away_epa', 'total_home_rush_epa','total_away_rush_epa', 'total_home_pass_epa', 'total_away_pass_epa', 'air_epa', 'yac_epa', 'comp_air_epa', 'total_home_comp_air_epa', 'total_away_comp_air_epa', 'total_home_comp_yac_epa', 'total_away_comp_yac_epa', 'total_home_raw_air_epa', 'total_away_raw_air_epa', 'total_home_raw_yac_epa', 'total_away_raw_yac_epa', 'wp', 'def_wp', 'home_wp', 'away_wp', 'wpa', 'home_wp_post', 'away_wp_post', 'total_home_rush_wpa', 'total_away_rush_wpa', 'total_home_pass_wpa', 'total_away_pass_wpa', 'air_wpa', 'yac_wpa', 'comp_air_wpa', 'comp_yac_wpa', 'total_home_comp_air_wpa', 'total_away_comp_air_wpa', 'total_home_comp_yac_wpa', 'total_away_comp_yac_wpa', 'total_home_raw_air_wpa', 'total_away_raw_air_wpa', 'total_home_raw_yac_wpa', 'total_away_raw_yac_wpa']

# a lot of these need to be re categorized
# you can easily create a "down" list
# there's a bunch of defensive stats in there that you can add to the defensive column, samewith punts, safety and what not
miscellaneous_plays = ['punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty', 'third_down_converted', 'third_down_failed', 'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'touchback', 'interception', 'fumble_forced', 'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety', 'penalty', 'tackled_for_loss', 'fumble_lost', 'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown', 'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', "complete_pass", "assisted_tackle", "lateral_reception", "lateral_rush", "lateral_return", "lateral_recovery"]

# automate
kickoff_punt_info = ['punt_inside_twenty', 'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone', 'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'own_kickoff_recovery', 'own_kickoff_recovery_td']

# Add to passer/rusher/reception/kicker and so on columns
player_info = ["passer_player_id", "passer_player_name", "receiver_player_id", "receiver_player_name", "rusher_player_id", "rusher_player_name", "lateral_receiver_player_id", "lateral_receiver_player_name", "lateral_rusher_player_id", "lateral_rusher_player_name", "lateral_sack_player_id", "lateral_sack_player_name", "lateral_sack_player_name", "interception_player_id", "interception_player_name", "lateral_interception_player_id", "lateral_interception_player_name", "punt_returner_player_id", "punt_returner_player_name", "lateral_punt_returner_player_id", "lateral_punt_returner_player_name", "kickoff_returner_player_name", "kickoff_returner_player_id", "lateral_kickoff_returner_player_id", "lateral_kickoff_returner_player_name", "punter_player_id", "punter_player_name", "kicker_player_name", "kicker_player_id", "own_kickoff_recovery_player_id", "own_kickoff_recovery_player_name", "blocked_player_id", "tackle_for_loss_1_player_id", "tackle_for_loss_1_player_name", "tackle_for_loss_2_player_id", "tackle_for_loss_2_player_name", "qb_hit_1_player_id", "qb_hit_1_player_name", "qb_hit_2_player_id", "qb_hit_2_player_name", "forced_fumble_player_1_player_id", "forced_fumble_player_1_player_name", "forced_fumble_player_2_player_id", "forced_fumble_player_2_player_name", "solo_tackle_1_player_id", "solo_tackle_2_player_id", "solo_tackle_1_player_name", "solo_tackle_2_player_name", "assist_tackle_1_player_id", "assist_tackle_1_player_name", "assist_tackle_2_player_id", "assist_tackle_2_player_name", "assist_tackle_3_player_id", "assist_tackle_3_player_name",  "assist_tackle_4_player_id", "assist_tackle_4_player_name",  "pass_defense_1_player_id", "pass_defense_1_player_name", "pass_defense_2_player_id", "pass_defense_2_player_name", "fumbled_1_player_id", "fumbled_1_player_name", "fumbled_2_player_id", "fumbled_2_player_name", "fumble_recovery_1_player_id", "fumble_recovery_1_player_name",  "fumble_recovery_2_player_id", "fumble_recovery_2_player_name"]

# automate, why is there an empty column
penalty_info = ["penalty_team", "penalty_player_id", "penalty_player_name", "penalty_yards", "replay_or_challenge", "replay_or_challenge_result", "penalty_type", ""]

# automate
defensive_points = ["defensive_two_point_attempt", "defensive_extra_point_attempt", "defensive_extra_point_conv"]

# Create a final list which includes all elements not currently grouped. Then examine the list and see if you
# can recategorize some elements

# giant list = sum of all list
# df.columns
# remaining = []

In [ ]:
# We often use list comprehensions when building out list out of conditions
# They have a better performance that for loops, and provide for neater code

prob_cols = [col for col in df.columns if 'prob' in col]

# The above line does the same thing as the code blow below

In [ ]:
# delete
probability_info2 = []

for col in data.columns:
    if 'prob' in col:
        probability_info2.append(col)

# do the same for other catagories 

## Step 4

In [ ]:
'''
Change Header
Remove redundant code
use the new filtering method to create the duplicate df and re add it
'''

In [ ]:
# delete
# finds the row index where each pass play is located

pass_ls = []
pass_row_num = 0
for i in data["play_type"]:
    if i == "pass":
        pass_ls.append(pass_row_num)
    pass_row_num += 1

    


In [ ]:
# delete


# uses the row indecies to add the pass plays to a new dataframe

pass_rows = pd.DataFrame()

for i in pass_ls:
    new_row = data.iloc[i]  
    pass_rows = pass_rows.append(new_row, ignore_index=True)

In [ ]:
# Modify
data[(data['play_type']=="pass")].head()
new_df['play_type'] = 'reception'
df  = pd.concat([df1,df2])

## Step 5

In [ ]:
'''
Change Header
Remove redundant code
The idea here is good but there's way too much code
I written a code that perform step 5 in a different way. Feel free to use it, or use its structure to recreate your code
'''

### Roy's Step 5

In [ ]:

data["Playmaker_id"] = ""

play_ls = []
id_dict = {}
row_num = 0

for i in data["play_type"]:
    
    if i == "pass":
        curr_id = data["passer_player_id"][row_num]
        if (data["passer_player_id"][row_num] not in play_ls):
            id_dict[curr_id] = data["passer_player_name"][row_num]
        play_ls.append(data["passer_player_id"][row_num])
        
        
    elif i == "kickoff":
        curr_id = data["kicker_player_id"][row_num]
        if (data["kicker_player_id"][row_num] not in play_ls):
            id_dict[curr_id] = data["kicker_player_name"][row_num]
        play_ls.append(data["kicker_player_id"][row_num])
        
        
    elif i == "run" or i == "qb_kneel":
        curr_id = data["rusher_player_id"][row_num]
        if (data["rusher_player_id"][row_num] not in play_ls):
            id_dict[curr_id] = data["rusher_player_name"][row_num]
        play_ls.append(data["rusher_player_id"][row_num])
        
        
    elif i == "punt":
        curr_id = data["punter_player_id"][row_num]
        if (data["punter_player_id"][row_num] not in play_ls):
            id_dict[curr_id] = data["punter_player_name"][row_num]
        play_ls.append(data["punter_player_id"][row_num])
        
        
    elif i == "field_goal":
        curr_id = data["kicker_player_id"][row_num]
        if (data["kicker_player_id"][row_num] not in play_ls):
            id_dict[curr_id] = data["kicker_player_name"][row_num]
        play_ls.append(data["kicker_player_id"][row_num])
        
        
    elif i == "extra_point":
        curr_id = data["kicker_player_id"][row_num]
        if (data["kicker_player_id"][row_num] not in play_ls):
            id_dict[curr_id] = data["kicker_player_name"][row_num]
        play_ls.append(data["kicker_player_id"][row_num])
        
        
    else:
        play_ls.append("N/A")
    
           
    row_num += 1


data["Playmaker_id"] = play_ls

data




In [ ]:
# list of all the player ids in specified columns
passer_ls = []
kickers_ls = []
rusher_ls = []
punter_ls = []

# appends all the player id in the column to corresponding list
for passers in data["passer_player_id"]:
    passer_ls.append(passers)
    
for kickers in data["kicker_player_id"]:
    kickers_ls.append(kickers)
    
for rusher in data["rusher_player_id"]:
    rusher_ls.append(rusher)
    
for punter in data["punter_player_id"]:
    punter_ls.append(punter)

# these lists will contain the names of the players in the order they appear in the dataframe
passer_names = []
kickers_names = []
rusher_names = []
punter_names = []

# the player IDs are used to retrieve the corresponding name from the dictionary
# in the dictionary one name corresponds to one ID which ensures uniqueness

for i in passer_ls:
    passer_names.append(id_dict[i])

for i in kickers_ls:
    kickers_names.append(id_dict[i])

for i in rusher_ls:
    rusher_names.append(id_dict[i])

for i in punter_ls:
    punter_names.append(id_dict[i])

# the list of names formed above are used unique as only one name is assigned per player, and the list is assigned to each column
data["passer_player_name"] = passer_names
data["kicker_player_name"] = kickers_names
data["rusher_player_name"] = rusher_names
data["punter_player_name"] = punter_names





In [ ]:
check_list = []
player_count = 0
ID_count = 0
for i in data["passer_player_name"].unique():
    player_count += 1
    check_list.append(i)
for i in data["kicker_player_name"].unique():
#     print(i)
    player_count += 1
    check_list.append(i)
for i in data["rusher_player_name"].unique():
    player_count += 1
    check_list.append(i)
for i in data["punter_player_name"].unique():
    player_count += 1
    check_list.append(i)

# print(player_count)
print(len(id_dict))
# print(len(check_list))
new_frame = pd.DataFrame()
new_frame['name'] = check_list

for i in new_frame['name'].unique():
#     print(i)
    ID_count += 1

print(ID_count)


# print(check_list)



# for i in new_frame['ID'].unique():
#     ID_count += 1

# print(ID_count)

new_frame
    

### Steven's Step 5

#### S5.1 Define Play Maker

In [ ]:
# Create play maker column
# you will need to add a receiver segments as well
df['play_maker'] = np.where(df['play_type']=='rush',df['rusher'],np.NaN)
df['play_maker'] = np.where(df['play_type']=='pass',df['passer'],df['play_maker'])
df['play_maker'] = np.where((df['play_type']=='extra_point')|(df['play_type']=='field_goal'),df['kicker'],df['play_maker'])

In [ ]:
# Add in corresponding play maker ID
df['play_maker_id'] = np.where(df['play_type']=='rush',df['rusher_player_id'],np.NaN)
df['play_maker_id'] = np.where(df['play_type']=='pass',df['passer_player_id'],df['play_maker_id'])
df['play_maker_id'] = np.where((df['play_type']=='extra_point')|(df['play_type']=='field_goal'),df['kicker_player_id'],df['play_maker_id'])

# Now that we have a single column to identify play makers, it is a lot easier to check for ID uniqueness

#### S5.2 Identify Non Unique Player Names

In [ ]:
# Create a data frame that contains the unique count of each player name under a given ID
# Filter on the IDs that correspond to more than one name

nunique_id = df[df.groupby(['play_maker_id'])['play_maker'].transform('nunique') > 1]['play_maker_id'].unique()

In [ ]:
# Identify all the duplicate names

df[df['play_maker_id'].isin(nunique_id)]['play_maker'].unique()

In [ ]:
# Create a dictionary of what the corrected names should be

name_corrections = {'D.Chark Jr.': 'D.Chark',
'Jos.Allen':'J.Allen',
'M.Ingram II': 'M.Ingram',
'A.Levine Sr.': 'A.Levine',
'R.Griffin III': 'R.Griffin',
'G.Minshew II':'G.Minshew',
'B.Snell Jr.':'B.Snell', 
'Tr.Edmunds':'T.Edmunds',
'R.James Jr.': 'R.James',
'J.Ross III':'J.Ross',
'W.Snead IV':'W.Snead', 
'M.Jones Jr.': 'M.Jones', 
'M.Sanu Sr.':'M.Sanu', 
'O.Beckham Jr.':'O.Beckham', 
'P.Dorsett II':'P.Dorsett'}

#### S5.3 Correct Name Uniqueness

In [ ]:
# Create a function to correct the typos

def typo_correction(name):
    if name in name_corrections.keys():
        return name_corrections[name]
    else:
        return name

In [ ]:
# apply the function to the dataframe

df['play_maker'] = df['play_maker'].apply(typo_correction)

#### S5.4 New Attempt

In [53]:
test = df[df['passer_player_id'].isin(nunique_id)].groupby(['passer_player_id','passer_player_name']).size().reset_index()
test.set_index('passer_player_id', inplace=True)

In [56]:
d = {player_id:(test.loc[player_id,'passer_player_name'][0],test.loc[player_id,'passer_player_name'][1]) for player_id in nunique_id}

In [63]:
test2 = df.copy()

In [64]:
test2.set_index('passer_player_id', inplace=True)

In [65]:
for dup in d.keys():
    test2.loc[dup,'passer_player_name'] = d[dup][0]

## Step 6

In [ ]:
'''
Add more descriptive titles
Re create this step using the where statements that i've used in the previous code
'''

In [ ]:
# # step 6:
relevance_ls = []
data["Flag"] = ""
# row_num = 1
for i in data["play_type"]:
    if i == "run" or i == "pass":
        relevance_ls.append(1)
#         data["Play_relevance"] = 1
    else:
        relevance_ls.append(0)
#         data["Play_relevance"] = 0

# Add field goal, extra points, qb_kneels and reception when done with step 4
data["Flag"] = relevance_ls

    

In [ ]:
# step 7

In [ ]:
# step 8